<a href="https://colab.research.google.com/github/snaiws/NLP_project/blob/RoBERTa%2Flarge/STS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semantic Textual Similarity for Korean



## Import

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 14.3 MB/s 
     |████████████████████████████████| 596 kB 57.1 MB/s 
     |████████████████████████████████| 86 kB 6.4 MB/s 
     |████████████████████████████████| 6.6 MB 61.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [1]:
import os
import sys
import re
import pandas as pd
import numpy as np 
import copy
import time
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader,  RandomSampler, SequentialSampler, random_split
from torch.nn.utils import clip_grad_norm_

from transformers import RobertaTokenizer, RobertaModel
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, pipeline
import transformers
from transformers import AdamW, get_linear_schedule_with_warmup

from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler, random_split
from transformers import BertTokenizer
from transformers import AutoTokenizer


In [2]:
# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)


# available GPUs : 1
GPU name : Tesla T4
cuda


In [3]:
# seed
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [5]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [6]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [7]:
!git clone https://github.com/snaiws/NLP_project

fatal: destination path 'NLP_project' already exists and is not an empty directory.


In [10]:
cd /content/drive/MyDrive/NLP_project

/content/drive/MyDrive/NLP_project


### ARGUMENT and Model

In [9]:
print(os.listdir('/content/drive/MyDrive/NLP_project/CustomModels'))

['BERT_baseline.py', 'BERT_seq.py', '__pycache__']


In [4]:
import torch.nn as nn
from transformers import AutoModel

class RoBERTa_large(nn.Module):

    def __init__(self, hidden_size: int, n_label: int, freeze_base: bool = False):
        super(RoBERTa_large, self).__init__()

        self.roberta = AutoModel.from_pretrained("klue/roberta-large")

        if freeze_base:
            for param in self.bert.parameters():
                param.requires_grad=False

        dropout_rate = 0.1
        linear_layer_hidden_size = 6

        self.classifier = nn.Sequential(
        nn.Linear(hidden_size, linear_layer_hidden_size),
        nn.ReLU(),
        nn.Dropout(dropout_rate),
        nn.Linear(linear_layer_hidden_size, n_label)
        )

    

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None):

        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        last_hidden_states = outputs[0] # last hidden states (batch_size, sequence_len, hidden_size)
        cls_token_last_hidden_states = last_hidden_states[:,0,:] # (batch_size, first_token, hidden_size)

        logits = self.classifier(cls_token_last_hidden_states)

        return logits

In [5]:
# argument setting
train_batch_size = 20
eval_batch_size = 64
epochs=25
patience=2

loss_fct = nn.CrossEntropyLoss()
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")
#tokenizer = BertTokenizer.from_pretrained("klue/roberta-small")
#tokenizer = BertTokenizer.from_pretrained("klue/roberta-base")
#tokenizer = BertTokenizer.from_pretrained("klue/roberta-large")
customModel = RoBERTa_large(hidden_size=1024, n_label=6)
customOptimizer = AdamW(
    customModel.parameters(), # update 대상 파라미터를 입력
    lr=2.335281794951567e-05,
    eps=1e-8
)

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it f

### Given data

In [11]:
# 파일 내 json 불러오기
df_train0 = pd.read_json('./data/klue-sts-v1.1_train.json')
df_test0 = pd.read_json('./data/klue-sts-v1.1_dev.json')

In [15]:
# shape
df_train0.shape, df_test0.shape

((11668, 6), (519, 6))

### Collected data

### target balence

In [ ]:
df_train0['labels'].map(lambda x: x['real-label']).hist(bins=50)

In [ ]:
df_train0['labels'].map(lambda x: x['binary-label']).value_counts().plot(kind='bar')

### duplicates

In [ ]:
#데이터 중복
df_train0[df_train0.duplicated(['sentence1','sentence2'],keep=False)==True]

In [ ]:
df_test0[df_test0.duplicated(['sentence1','sentence2'],keep=False)==True]

In [ ]:
pd.concat([df_train0.drop_duplicates(['sentence1','sentence2']),df_test0]).duplicated(['sentence1','sentence2']).sum()

In [ ]:
df_train0.isna().sum().sum()+df_test0.isna().sum().sum()

## Data preprocess

### Preprocess

In [6]:
def check_st(text):
  text = re.sub(r"[^ㄱ-힣0-9\s]","",text)
  return text

def base_preprocess(train, test):
    train = train.drop_duplicates(['sentence1','sentence2']).reset_index(drop=True)
    s1 = train['sentence1'].map(lambda x: check_st(x))
    s2 = train['sentence2'].map(lambda x: check_st(x))
    real = train['labels'].map(lambda x: int(x['real-label']))
    binary = train['labels'].map(lambda x: int(x['binary-label']))
    train = pd.concat([s1,s2,real,binary],axis=1)
    train.columns = ['sentence1','sentence2','real','binary']
    train['sentence'] = train.apply(lambda x: x['sentence1']+' [SEP] '+x['sentence2'],axis=1)
    s1 = test['sentence1'].map(lambda x: check_st(x))
    s2 = test['sentence2'].map(lambda x: check_st(x))
    real = test['labels'].map(lambda x: int(x['real-label']))
    binary = test['labels'].map(lambda x: int(x['binary-label']))
    test = pd.concat([s1,s2,real,binary],axis=1)
    test.columns = ['sentence1','sentence2','real','binary']
    test['sentence'] = test.apply(lambda x: x['sentence1']+' [SEP] '+x['sentence2'],axis=1)
    return train, test

In [12]:
df_train, df_test = base_preprocess(df_train0,df_test0)

In [ ]:
df_train.head()

### Augmentation

### Token length

In [ ]:
def tknz(text):
    x = tokenizer(
            text,
            add_special_tokens=True,
            truncation=True, # max_length를 넘는 문장은 이 후 토큰을 제거함
            max_length=512,
            return_tensors='pt' # 토크나이즈된 결과 값을 텐서 형태로 반환
        )
    return x

In [ ]:
df_train['sentence1'].map(lambda x: tknz(x).input_ids.shape[1]).value_counts().sort_index().plot(kind='bar',figsize=(10,5))

In [ ]:
df_train['sentence2'].map(lambda x: tknz(x).input_ids.shape[1]).value_counts().sort_index().plot(kind='bar',figsize=(10,5))

In [ ]:
df_train['sentence'].map(lambda x: tknz(x).input_ids.shape[1]).value_counts().sort_index().plot(kind='bar',figsize=(20,5))

### Dataset

In [13]:
# Custom Dataset
class CustomDataset(Dataset):
    """
    - input_data: list of string
    - target_data: list of int
    """
    
    def __init__(self, input_data:list, target_data:list) -> None:
        self.X = input_data
        self.Y = target_data
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.Y[index]

In [14]:
train_dataset = CustomDataset(df_train.sentence.to_list(), df_train.real.to_list())
test_dataset = CustomDataset(df_test.sentence.to_list(), df_test.real.to_list())

### Split Data

In [15]:
n_sample = df_train.shape[0] # train 전체 길이
n_train = int(n_sample*0.9)
n_valid = n_sample-n_train
train_dataset, valid_dataset = random_split(train_dataset, [n_train, n_valid],generator=torch.Generator().manual_seed(seed))

In [21]:
print(f"Train Dataset len: {len(train_dataset)}")
print(f"Valid Dataset len: {len(valid_dataset)}")
print(f"Test Dataset len: {len(test_dataset)}")

Train Dataset len: 10494
Valid Dataset len: 1167
Test Dataset len: 519


## Plan

### Data Loader

In [16]:
# Custom collate_fn 
def BERT_base_collate_fn(batch):
    """
    한 배치 내 문장들을 tokenizing 한 후 텐서로 변환함. 
    이때, dynamic padding (즉, 같은 배치 내 토큰의 개수가 동일할 수 있도록, 부족한 문장에 [PAD] 토큰을 추가하는 작업)을 적용
    
    한 배치 내 레이블(target)은 텐서화 함.
    
    - batch: list of tuples (input_data(string), target_data(int))
    """
    input_list, target_list = [], []

    for _input, _target in batch:
        input_list.append(_input)
        target_list.append(_target)
    
    tensorized_input = tokenizer(
        input_list,
        add_special_tokens=True,
        padding="longest", # 배치내 가장 긴 문장을 기준으로 부족한 문장은 [PAD] 토큰을 추가
        truncation=True, # max_length를 넘는 문장은 이 후 토큰을 제거함
        max_length=512,
        return_tensors='pt' # 토크나이즈된 결과 값을 텐서 형태로 반환
    )
    
    tensorized_label = torch.tensor(target_list)
    
    return tensorized_input, tensorized_label

In [17]:
def init_data_loader(train_dataset,valid_dataset,test_dataset):
    train_dataloader = DataLoader(
        train_dataset,
        batch_size = train_batch_size,
        sampler = RandomSampler(train_dataset,generator=torch.Generator().manual_seed(seed)),
        collate_fn = BERT_base_collate_fn
    )

    valid_dataloader = DataLoader(
        valid_dataset,
        batch_size = eval_batch_size,
        sampler = SequentialSampler(valid_dataset),
        collate_fn = BERT_base_collate_fn
    )

    test_dataloader = DataLoader(
        test_dataset,
        batch_size = eval_batch_size,
        sampler = SequentialSampler(test_dataset),
        collate_fn = BERT_base_collate_fn
    )
    print(f"Train dataloader # steps: {len(train_dataloader)}")
    print(f"Valid dataloader # steps: {len(valid_dataloader)}")
    print(f"Test dataloader # steps: {len(test_dataloader)}")
    return train_dataloader, valid_dataloader, test_dataloader

In [18]:
train_dataloader, valid_dataloader, test_dataloader = init_data_loader(train_dataset,valid_dataset,test_dataset)

Train dataloader # steps: 525
Valid dataloader # steps: 19
Test dataloader # steps: 9


### Initializer

In [19]:
def initializer(model, optimizer, train_dataloader, epochs=2):
    """
    모델, 옵티마이저, 스케쥴러 초기화
    """
    total_steps = len(train_dataloader) * epochs
    print(f"Total train steps with {epochs} epochs: {total_steps}")

    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps = 0, # 여기서는 warmup을 사용하지 않는다.
        num_training_steps = total_steps
    )

    return model, optimizer, scheduler

### Early stopping

In [20]:
class EarlyStopping:
    """주어진 patience 이후로 validation loss가 개선되지 않으면 학습을 조기 중지"""
    def __init__(self, patience=1, verbose=False, delta=0, path='checkpoint.pt'):
        """
        Args:
            patience (int): validation loss가 개선된 후 기다리는 기간
                            Default: 7
            verbose (bool): True일 경우 각 validation loss의 개선 사항 메세지 출력
                            Default: False
            delta (float): 개선되었다고 인정되는 monitered quantity의 최소 변화
                            Default: 0
            path (str): checkpoint저장 경로
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = os.path.abspath(os.curdir)

    def __call__(self, val_loss, model, optimizer, scheduler, epoch):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model, optimizer, scheduler, epoch, val_loss)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model, optimizer, scheduler, epoch, val_loss)
            self.counter = 0

    def save_checkpoint(self, model, optimizer, scheduler, epoch, loss):
        file_name = f'{self.path}/model.ckpt.best'
        
        torch.save(
            {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'loss' : loss
            }, 
            file_name
        )
      
        print(f"Saving epoch {epoch} checkpoint at {file_name}")

### Train

In [21]:
def train_model(model, train_dataloader, valid_dataloader=None, epochs=1):
    # early_stopping object의 초기화
    early_stopping = EarlyStopping(patience = patience, verbose = True)
    
    # train_dataloaer 학습을 epochs만큼 반복
    for epoch in range(1, epochs + 1):
        print(f"*****Epoch {epoch} Train Start*****")
        
        # 배치 단위 평균 loss와 총 평균 loss 계산하기위해 변수 생성
        total_loss, batch_loss, batch_count = 0,0,0
    
        # model을 train 모드로 설정 & device 할당
        model.train()
        model.to(device)
        
        # data iterator를 돌면서 하나씩 학습
        for step, batch in enumerate(train_dataloader):
            batch_count+=1
            
            # tensor 연산 전, 각 tensor에 device 할당
            batch = tuple(item.to(device) for item in batch)
        
            batch_input, batch_label = batch
        
            # batch마다 모델이 갖고 있는 기존 gradient를 초기화
            model.zero_grad()
        
            # forward
            logits = model(**batch_input)
            # loss
            loss = loss_fct(logits, batch_label)
            batch_loss += loss.item()
            total_loss += loss.item()
        
            # backward -> 파라미터의 미분(gradient)를 자동으로 계산
            loss.backward()
            
            # gradient clipping 적용 
            clip_grad_norm_(model.parameters(), 1.0)
            
            # optimizer & scheduler 업데이트
            optimizer.step()
            scheduler.step()

            # 배치 10개씩 처리할 때마다 평균 loss와 lr를 출력
            if (step % 10 == 0 and step != 0):
                learning_rate = optimizer.param_groups[0]['lr']
                print(f"Epoch: {epoch}, Step : {step}, LR : {learning_rate}, Avg Loss : {batch_loss / batch_count:.4f}")

                # reset 
                batch_loss, batch_count = 0,0
        
        print(f"Epoch {epoch} Total Mean Loss : {total_loss/(step+1):.4f}")
        print(f"*****Epoch {epoch} Train Finish*****\n")
        
        if valid_dataloader is not None:
            print(f"*****Epoch {epoch} Valid Start*****")
            valid_loss, valid_acc = validate(model, valid_dataloader)
            print(f"Epoch {epoch} Valid Loss : {valid_loss:.4f} Valid Acc : {valid_acc:.2f}")
            print(f"*****Epoch {epoch} Valid Finish*****\n")

        # early_stopping는 validation loss가 감소하였는지 확인이 필요하며,
        #  만약 감소하였을경우 현제 모델을 checkpoint로 만든다.
        early_stopping(valid_loss, model, optimizer, scheduler, epoch)

        if early_stopping.early_stop:
            print("Early stopping")
            break
            
            
    print("Train Completed. End Program.")

### Validate

In [22]:
def validate(model, valid_dataloader):
   
    # 모델을 evaluate 모드로 설정 & device 할당
    model.eval()
    model.to(device)
    
    total_loss, total_acc= 0,0
        
    for step, batch in enumerate(valid_dataloader):
        
        # tensor 연산 전, 각 tensor에 device 할당
        batch = tuple(item.to(device) for item in batch)
            
        batch_input, batch_label = batch
            
        # gradient 계산하지 않음
        with torch.no_grad():
            logits = model(**batch_input)
            
        # loss
        loss = loss_fct(logits, batch_label)
        total_loss += loss.item()
        
        # accuracy
        probs = F.softmax(logits, dim=1)
        preds = torch.argmax(probs, dim=1).flatten()
        acc = (preds == batch_label).cpu().numpy().mean()
        total_acc+=acc
    
    total_loss = total_loss/(step+1)
    total_acc = total_acc/(step+1)*100

    return total_loss, total_acc
    

## Train

In [53]:
# reset gpu cache
import gc
gc.collect()
torch.cuda.empty_cache()

In [54]:
# train
y, M, d, h, m, s, w, yd, isdst = time.gmtime()
current_time = f'{y}.{M}.{d} {h+9}:{m}:{s}'
print(current_time)
torch.cuda.manual_seed_all(seed)
train_dataloader, valid_dataloader, test_dataloader = init_data_loader(train_dataset,valid_dataset,test_dataset)
model, optimizer, scheduler = initializer(customModel, customOptimizer, train_dataloader, epochs)
train_model(model, train_dataloader, valid_dataloader, epochs)

2022.6.1 22:29:0
Train dataloader # steps: 525
Valid dataloader # steps: 19
Test dataloader # steps: 9
Total train steps with 25 epochs: 13125
*****Epoch 1 Train Start*****
Epoch: 1, Step : 10, LR : 2.3333246063996078e-05, Avg Loss : 1.6692
Epoch: 1, Step : 20, LR : 2.3315453440796446e-05, Avg Loss : 1.5829
Epoch: 1, Step : 30, LR : 2.3297660817596814e-05, Avg Loss : 1.2966
Epoch: 1, Step : 40, LR : 2.3279868194397185e-05, Avg Loss : 1.1838
Epoch: 1, Step : 50, LR : 2.3262075571197553e-05, Avg Loss : 1.1550
Epoch: 1, Step : 60, LR : 2.3244282947997924e-05, Avg Loss : 0.9939
Epoch: 1, Step : 70, LR : 2.3226490324798288e-05, Avg Loss : 1.1784
Epoch: 1, Step : 80, LR : 2.320869770159866e-05, Avg Loss : 1.1007
Epoch: 1, Step : 90, LR : 2.3190905078399027e-05, Avg Loss : 1.1100
Epoch: 1, Step : 100, LR : 2.31731124551994e-05, Avg Loss : 1.1719
Epoch: 1, Step : 110, LR : 2.3155319831999766e-05, Avg Loss : 1.1188
Epoch: 1, Step : 120, LR : 2.3137527208800137e-05, Avg Loss : 1.1511
Epoch: 1, S

KeyboardInterrupt: ignored

## Evaluation

### Load checkpoint

In [55]:
checkpoint = torch.load(f'./model.ckpt.best')

In [56]:
checkpoint.keys()

dict_keys(['epoch', 'model_state_dict', 'optimizer_state_dict', 'scheduler_state_dict', 'loss'])

In [57]:
epochs=1
model, optimizer, scheduler = initializer(customModel, customOptimizer, train_dataloader, epochs)

Total train steps with 1 epochs: 525


In [58]:
model.load_state_dict(checkpoint["model_state_dict"])

<All keys matched successfully>

### predict and evaluate

In [23]:
def predict(model, test_dataloader):
    """
    test_dataloader의 label별 확률값과 실제 label 값을 반환
    """

    model.eval()
    model.to(device)

    all_logits = []
    all_labels = []

    for step, batch in enumerate(test_dataloader):
        print(f"{step}/{len(test_dataloader)}")
        
        batch_input, batch_label = batch
        
        batch_input = batch_input.to(device)
        
        with torch.no_grad():
            logits = model(**batch_input)
            all_logits.append(logits)
        all_labels.extend(batch_label)

    all_logits = torch.cat(all_logits, dim=0)
    probs = F.softmax(all_logits, dim=1).cpu().numpy()
    all_labels = np.array(all_labels)

    return probs, all_labels


In [60]:
probs, labels = predict(model, test_dataloader)

0/9
1/9
2/9
3/9
4/9
5/9
6/9
7/9
8/9


In [61]:
pred = list(map(np.argmax,probs))

In [24]:
from sklearn.metrics import f1_score, accuracy_score
from scipy.stats import pearsonr
def evaluate(pred, test):
    if len(set(pred))==2:
        pred_binary = pred
        test_binary_label = test
    else:
        pred_binary = list(map(lambda x: 1 if x>=3 else 0, pred))
        test_binary_label = list(map(lambda x: 1 if x>=3 else 0, test))
    print(f'acc : {accuracy_score(test_binary_label, pred_binary)}')
    print(f'f1 : {f1_score(test_binary_label, pred_binary)}')
    print(f'pearson : {pearsonr(test, pred)}')
    return f1_score(test_binary_label, pred_binary)
    

In [63]:
evaluate(pred, labels)

acc : 0.8169556840077071
f1 : 0.8197343453510436
pearson : (0.8532410401789287, 2.762453970921971e-148)


0.8197343453510436

### Save outputs

In [ ]:
output = pd.DataFrame(labels,columns=['pred_real_label'])
filename = input()
output.to_csv(f'{filename}.csv')

BERT_seq


In [ ]:
torch.save(model.state_dict(), 'model_weights.pth')

## Load model weights

In [ ]:
model = Bert_baseline(hidden_size=768, n_label=6)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model.load_state_dict(torch.load('model_weights.pth'))
model.eval()
probs, labels = predict(model, test_dataloader)

0/9
1/9
2/9
3/9
4/9
5/9
6/9
7/9
8/9


In [ ]:
pred = list(map(np.argmax,probs))
evaluate(pred, labels)

acc : 0.7533718689788054
f1 : 0.7697841726618705
pearson : (0.7789983354145806, 7.071859924703616e-107)


## Hyperparameter tuning

In [64]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 32.6 MB/s 
     |████████████████████████████████| 81 kB 10.8 MB/s 
     |████████████████████████████████| 209 kB 84.3 MB/s 
     |████████████████████████████████| 78 kB 9.3 MB/s 
     |████████████████████████████████| 146 kB 5.7 MB/s 
     |████████████████████████████████| 112 kB 70.9 MB/s 
     |████████████████████████████████| 49 kB 7.8 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=8f4f1ccf98f0bd347a9426e3321de26f38fe8ef493725a8b687cea320cc39114
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [25]:
import optuna

In [26]:
def optuna_train(model, optimizer):
    model.train()
    model.to(device)
    for step, batch in enumerate(train_dataloader):
        batch = tuple(item.to(device) for item in batch)
        batch_input, batch_label = batch
        model.zero_grad()
        logits = model(**batch_input)
        loss = loss_fct(logits, batch_label)
        loss.backward()
        clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

In [27]:
def objective(trial):  # `trial` is an object passed by Optuna.
    lr = trial.suggest_float("lr",1e-6,1e-3,log=True)


    train_dataloader = DataLoader(
        train_dataset,
        batch_size = train_batch_size,
        sampler = RandomSampler(train_dataset),
        collate_fn = BERT_base_collate_fn
        )
    model = customModel
    optimizer = torch.optim.Adam(
        model.parameters(), lr = lr
        )
    val_list = []
    for epoch in range(4):
        optuna_train(model,optimizer)
        valid_loss, valid_acc = validate(model, valid_dataloader)
        print(f'val_los : {valid_loss}')
        trial.report(valid_loss, step = epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
        val_list.append(valid_loss)
    return min(val_list)

In [28]:
# reset gpu cache
import gc
gc.collect()
torch.cuda.empty_cache()

In [29]:
study = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.TPESampler(seed=seed),
    pruner=optuna.pruners.HyperbandPruner(
        min_resource=1, 
        max_resource=4, 
        reduction_factor=3
    ))
study.optimize(objective, n_trials=50)

[I 2022-06-01 13:58:25,074] A new study created in memory with name: no-name-9829399e-27b8-4b41-967e-623bad664e3d


KeyboardInterrupt: ignored

In [ ]:
print('Minimum objective value: ' + str(study.best_value))
print('Best parameter: ' + str(study.best_params))

ValueError: ignored